In [ ]:
######################################################
# Juypter
######################################################
%load_ext autoreload
%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))


###########################################################################
## Warnings
###########################################################################
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
warnings.filterwarnings("ignore", category=FutureWarning) 



################################################################################
## Music DB
################################################################################
from masterDBGate import masterDBGate
from mainDB import mainDB
from musicDBMap import musicDBMap
from masterDBMatchClass import masterDBMatchClass
from matchDBArtist import matchDBArtist
from dbArtistsMetadata import dbArtistsMetadata

from dbArtistsParseExtra import dbArtistsAssertExtra, dbArtistsExtra

######################################################
# Versions
######################################################
from timeUtils import clock, elapsed
import sys
print("Python: {0}".format(sys.version))
import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

# AllMusic

In [ ]:
from 

In [ ]:
from fileIO import fileIO
io = fileIO()
tabs = io.get("AllMusicTabs.p")

In [ ]:
from collections import Counter
cntr = Counter()
for idx,tab in tabs.iteritems():
    for key in tab.keys():
        cntr[key] += 1

In [ ]:
cntr

In [ ]:
from dbArtistsAllMusic import dbArtistsAllMusic
from time import sleep
from fsUtils import fileUtil
from timeUtils import timestat
dbAM = dbArtistsAllMusic()

N = 963
n = 0
ts = timestat("Downloading {0} Credits".format(N))
for i,(artistID,artistTabs) in enumerate(tabs.iteritems()):
    #for tab,tabRef in artistTabs.items():
    #    pass
    
    
    tabRef = artistTabs.get("Compositions")
    if tabRef is not None:
        #tabURL = "https://www.allmusic.com{0}/all".format(tabRef) ## Songs
        
        tabURL = "https://www.allmusic.com{0}".format(tabRef) ## Credits/Compositions
        tabSavename = dbAM.dutils.getArtistSavename(artistID, composition=True)
        if fileUtil(tabSavename).exists:
            continue
        
        print("="*150)
        print(artistID,'\t',tabURL,'\t',tabSavename)
        dbAM.dutils.downloadArtistURL(tabURL, tabSavename, force=False)
        print("")
        sleep(3)
        

    if (n+1) % 10 == 0:
        print("-"*150)
        print("")
        ts.update(n=n+1,N=N)
        print("")
        print("-"*150)
        print("")        
        
ts.stop()

# Credit/Song/Compositions

In [ ]:
%autoreload
from dbArtistsParse import dbArtistsPrimary
from dbArtistsParseCredit import dbArtistsCredit
from dbArtistsParseSong import dbArtistsSong
from dbArtistsParseComposition import dbArtistsComposition
from dbArtistsAllMusic import dbArtistsAllMusic

In [ ]:
from dbArtistsBase import dbArtistsBase
from fsUtils import isFile
from fileUtils import getBaseFilename
from timeUtils import timestat
from ioUtils import saveFile
from time import sleep
import urllib
from webUtils import getHTML


#################################################################################################################################
# Assert Song (Find Song For AllMusic)
#################################################################################################################################
class dbArtistsAssertSong(dbArtistsBase):
    def __init__(self, dbArtists):        
        super().__init__(dbArtists)
        self.setPrimary()
        self.dbArtists = dbArtists
        #self.dbSong = dbArtistsSong(dbArtists)
        print("dbArtistsAssertSong({0})".format(self.db))
        try:
            self.masterIgnoreData = self.getMasterIgnoreData()
            self.songIgnores = self.masterIgnoreData["AllMusic"]["Song"]
            print("  --> Found {0} AllMusic Song IDs To Ignore".format(len(self.songIgnores)))
        except:
            self.songIgnores = []
            #raise ValueError("Could not load AllMusic Credit Ignores data!")
        self.metadata = {}
                
    
    def getMetadata(self):
        return self.metadata
    
    def createSongMetadata(self, modVal=None):
        modVals = [modVal] if modVal is not None else range(100)
            
        ts = timestat("Creating AllMusic Song Metadata")
        for modVal in modVals:
            
            tsDBData = timestat("Finding Known Credit Artists For ModVal={0}".format(modVal))
            dbData = self.getDBData(modVal)
            dbArtistURLs    = {artistID: {"Name": artistData.artist.name, "URL": artistData.url.url} for artistID,artistData in dbData.items()}
            tsDBData.stop()            

            tsCredit = timestat("Finding Known Credit Artists From {0} Artists For ModVal={1}".format(len(dbArtistURLs), modVal))
            creditArtistIDs = {artistID: artistData for artistID,artistData in dbArtistURLs.items() if artistData["URL"] is not None and artistData["URL"].endswith("/credits")}
            tsCredit.stop()
            
            tsIgnore = timestat("Removing IDs To Ignore From {0} Primary Files For ModVal={0}".format(len(creditArtistIDs), modVal))
            availableArtistIDs = {artistID: artistData for artistID,artistData in creditArtistIDs.items() if artistID not in self.songIgnores}
            tsIgnore.stop()
            
            tsMeta = timestat("Finding Metadata For {0}/{1}/{2} Missing ArtistIDs for ModVal={3}".format(len(availableArtistIDs), len(creditArtistIDs), len(dbArtistURLs), modVal))
            self.metadata[modVal] = availableArtistIDs
            tsMeta.stop()
        ts.stop()

    
    def downloadUnknownArtistSongs(self):
        newIgnores = []
        for modVal,modValMetadata in self.metadata.items():
            N = len(modValMetadata)
            ts = timestat("Downloading {0} Unknown Song Files For ModVal={1}".format(N, modVal))
            for i,(artistID,artistIDData) in enumerate(modValMetadata.items()):
                savename = self.dutils.getArtistSavename(artistID, song=True)
                
                href   = artistIDData["URL"]
                artist = artistIDData["Name"]
                if isFile(savename):
                    continue

                ## Replace /credits with /songs
                href = "/".join(href.split('/')[:-1] + ["songs", "all"])
                    
                ## Create Full URL
                url = urllib.parse.urljoin(self.dbArtists.baseURL, href)
                print("\n")
                print("="*100)
                print("{0}/{1}:  [{2}] / [{3}]".format(i,N,artist,url))
                

                data, response = self.dutils.downloadURL(url)
                if response == 200:
                    bsdata = getHTML(data)
                    if len(bsdata.findAll("th", {"class": "title-composer"})) > 0:
                        print("  ---> Saving Data To {0}".format(savename))
                        saveFile(idata=data, ifile=savename)
                        sleep(3)
                        continue
                
                sleep(3)
                newIgnores.append(artistID)
                        
                
                if i == 20:
                    break
            ts.stop()
            
        print("New IDs To Ignore")
        print(newIgnores)
        tsUpdate = timestat("Adding {0} ArtistIDs To Master Song Ignore List".format(len(newIgnores)))
        self.updateMasterIgnoreSongData(newIgnores)
        tsUpdate.stop()  
 
        
#################################################################################################################################
# Song
#################################################################################################################################
class dbArtistsSong(dbArtistsBase):
    def __init__(self, dbArtists):        
        super().__init__(dbArtists)
        self.setSong()
        self.dbArtists = dbArtists
        
    def parse(self, modVal, expr, force=False, debug=False):
        ts = timestat("Parsing ModVal={0} Song Files".format(modVal))  
        
        tsFiles  = timestat("Finding Files To Parse")
        newFiles = self.getArtistSongFiles(modVal, expr, force=True)
        tsFiles.stop()

        N = len(newFiles)
        modValue = 500 if N >= 1000 else 100
        if N > 0:
            tsDB = timestat("Loading ModVal={0} DB Data".format(modVal))
            dbdata   = self.disc.getDBModValData(modVal) ## We do not want to overwrite other data
            tsDB.stop()
            
        newData  = 0
        tsParse = timestat("Parsing {0} New Song Files For ModVal={1}".format(N, modVal))
        return
        for i,ifile in enumerate(newFiles):
            if (i+1) % modValue == 0 or (i+1) == N:
                print("{0: <15}Parsing {1}".format("{0}/{1}".format(i+1,N), ifile))
            artistID = getBaseFilename(ifile)
            info     = self.artist.getData(ifile)
            
            currentKeys = []
            if dbdata.get(artistID) is not None:
                currentKeys = list(dbdata[artistID].media.media.keys())
            else:
                dbdata[artistID] = info
                newData += 1
                continue
            
            keys = list(set(list(info.media.media.keys()) + currentKeys))
            for k in keys:
                v = info.media.media.get(k)
                if v is None:
                    continue
                iVal  = {v2.code: v2 for v2 in v}
                dVal  = dbdata[artistID].media.media.get(k)
                if dVal is None:
                    Tretval = iVal
                else:
                    Tretval = {v2.code: v2 for v2 in dVal}
                    Tretval.update(iVal)
                dbdata[artistID].media.media[k] = list(Tretval.values())
            newData += 1
            
        if newData > 0:
            print("Saving {0} New Entries".format(newData))
            self.disc.saveDBModValData(idata=dbdata, modVal=modVal) ## We do not want to overwrite other data
            
        tsParse.stop()  

In [ ]:
disc       = masterDBGate().getDBDisc("AllMusic")
modValData = disc.getDBModValData('34')

In [ ]:
modValData['0000961234'].profile.get()

In [ ]:
%autoreload
from artistAllMusic import artistAllMusic
from dbArtistsAllMusic import dbArtistsAllMusic
art = artistAllMusic()
retval = art.getData('/Users/tgadfort/discogs/artists-allmusic/34/0000961234.p')
retval.profile.extra

In [ ]:
dbAP = dbArtistsPrimary(dbArtistsAllMusic())
dbAP.parse(modVal=34, expr='< 0 Days', force=False)

In [ ]:
dbAP = dbArtistsSong(dbArtistsAllMusic())
dbAP.parse(modVal=0, expr='< 0 Days', force=True)

In [ ]:
dbAP = dbArtistsCredit(dbArtistsAllMusic())
dbAP.parse(modVal=0, expr='< 0 Days', force=True)

In [ ]:
dbAP = dbArtistsComposition(dbArtistsAllMusic())
dbAP.parse(modVal=0, expr='< 0 Days', force=True)

# Metadata

In [ ]:
from masterDBGate import masterDBGate
mdbGate = masterDBGate()
for db in mdbGate.getDBs():
    if db in ["Discogs", "MusicBrainz"]:
        continue
    for modVal in range(100):
        dbAP = dbArtistsMetadata(mdbGate.getDBGate(db), modVal, create=True)

In [ ]:
dbAP.create

# Genius

In [ ]:
%autoreload
from dbArtistsGenius import dbArtistsGenius
from dbArtistsParse import dbArtistsPrimary
dbAP = dbArtistsPrimary(dbArtistsGenius())
dbAP.parse(0, expr="< 0 Days", force=True)

In [ ]:
x='KYUHYUN (규현)'
x[:7]

In [ ]:
art = artistGenius()
disc = masterDBGate().getDBDisc("Genius")
art.getData(disc.getRawFilename('79339603')).artist.name

In [ ]:
def isKorea(value):
    if regex.search(r'\p{IsHangul}', value):
        return True
    return False

def isThai(value):
    if regex.search(r'\p{IsThai}', value):
        return True
    return False

def isJapanChina(value):
    if regex.search(r'([\p{IsHan}\p{IsBopo}\p{IsHira}\p{IsKatakana}]+)', value):
        return True
    return False

def isRussia(value):
    if regex.search(r'\p{IsCyrillic}', value):
        return True
    return False

def isIsreal(value):
    if regex.search(r'\p{IsHebrew}', value):
        return True
    return False

def isLatin(value):
    if regex.search(r'\p{IsLatin}', value):
        return True
    return False


In [ ]:
isKorea(artistName)

In [ ]:
artistName='KYUHYUN (규현)'
import regex
def isLatin(value):
    if regex.search(r'\p{IsLatin}', value):
        return True
    return False


discogsPattern = regex.compile(r"\s\(\p{IsHangul}\)")
result = discogsPattern.search(artistName)
print(result)

discogsPattern = regex.compile(r"\(\p{IsHangul}\)")
result = discogsPattern.search(artistName)
print(result)

discogsPattern = regex.compile(r"\s\([\p{IsHangul}]+\)")
result = discogsPattern.search(artistName)
print(result)

In [ ]:
names = artistName.split()
if isLatin(names[0]) and names[-1].endswith(")"):
    print(names)

In [ ]:

    def isLatin(self, value):
        if regex.search(r'\p{IsLatin}', value):
            return True
        return False
    
    def splitNativeName(self, artistName):
        names = artistName.split()
        ## Rough syntax for non-Latin name with Latin name in ()
        if not self.isLatin(names[0]) and names[-1].endswith(")"):
            native = []
            latin  = []
            for i,name in enumerate(names):
                if name.startswith("("):
                    for latinname in names[i:]:
                        latin.append(latinname.replace("(", "").replace(")", ""))
                    break
                native.append(name)

            native = " ".join(native)
            latin  = " ".join(latin)
            return latin,native
        else:
            return artistName,None
            

In [ ]:
modData['79339603'].show()

In [ ]:
%load_ext autoreload
%autoreload
from dbArtistsGenius import dbArtistsGenius
from dbArtistsParse import dbArtistsPrimary, dbArtistsCredit, dbArtistsExtra
from dbArtistsParse import dbArtistsAssertExtra, dbArtistsUnofficial, dbArtistsAssertUnofficial, dbArtistsAssertCredit
modVals=range(39,40)
dbAP = dbArtistsPrimary(dbArtistsGenius())
from searchUtils import findExt
from fileUtils import getDirname, getBaseFilename
from fsUtils import setFile, moveFile, isFile, removeFile, setDir
from dbUtils import utilsGenius
utils = utilsGenius()
for modVal in modVals:
    files = findExt("/Volumes/Piggy/Discog/artists-genius/{0}/".format(modVal), ext=".p")
    print(modVal,len(files))
    for ifile in files:
        print('\t',ifile,'\t',end="")
        try:
            retval = dbAP.artist.getData(ifile)
        except:
            print("Bad File: {0}".format(ifile))
            removeFile(ifile)
            continue
        artistID = retval.get()["ID"]
        artistID = artistID.ID
        print(artistID,'\t',end="")
        
        artistURL = retval.get()["url"]
        artistURL = artistURL.url
        artistIDFromURL = utils.getArtistID(artistURL)
        print(artistIDFromURL,'\t',artistURL)
        
        try:
            int(artistID)
        except:
            print("Bad File: {0}".format(ifile))
            removeFile(ifile)
            continue
        #print(getDirname(ifile),getBaseFilename(ifile))
        srcFile = ifile
        dirval  = getDirname(getDirname(ifile))
        modval  = str(int(artistID) % 100)
        
        dstFile = setFile(setDir(dirval, modval), "{0}.p".format(artistID))
        if srcFile == dstFile:
            continue
        if isFile(dstFile):
            print("{0} already exists.".format(dstFile))
            retval2 = dbAP.artist.getData(dstFile)
            artistID2 = retval2.get()["ID"]
            artistID2 = artistID2.ID
            if artistID2 == artistID:
                print("\tRemove old file: {0}".format(ifile))
                removeFile(ifile)
            continue
            
            raise ValueError(srcFile,dstFile)
        print(srcFile,dstFile)
        moveFile(srcFile, dstFile)

In [ ]:
dbAP = dbArtistsPrimary(dbArtistsGenius())
dometa = dbAP.parse(0, previousDays=None, force=True, debug=True)

In [ ]:
ifile='/Volumes/Piggy/Discog/artists-genius/2/83329186.p'
retval = dbAP.artist.getData(ifile)
retval.show()

In [ ]:
ifile='/Volumes/Piggy/Discog/artists-genius/63/82287463.p'
retval = dbAP.artist.getData(ifile)
retval.show()

In [ ]:
dbAP.dutils.getArtistID('https://genius.com/artists/The-arcana', debug=True)

In [ ]:
#8606107
from webUtils import getHTML
ifile = getHTML("/Volumes/Piggy/Discog/artists-genius/0/49153000.p")
ifile

In [ ]:
retval = dbAP.artist.getData(ifile)
retval.show()

In [ ]:
data.keys()

In [ ]:
%load_ext autoreload
%autoreload
from dbArtistsGenius import dbArtistsGenius
from dbArtistsParse import dbArtistsPrimary, dbArtistsCredit, dbArtistsExtra
from dbArtistsParse import dbArtistsAssertExtra, dbArtistsUnofficial, dbArtistsAssertUnofficial, dbArtistsAssertCredit

In [ ]:
dbAP = dbArtistsPrimary(dbArtistsGenius())
dometa = dbAP.parse(56, previousDays=None, force=False, debug=True)

In [ ]:
for modVal in range(100):
    dbAP = dbArtistsPrimary(dbArtistsGenius())
    dometa = dbAP.parse(modVal, previousDays=None, force=True, debug=False)
    dometa = True
    if dometa:
        dbAP.createArtistMetadata(modVal)
        dbAP.createAlbumMetadata(modVal)

# AlbumOfTheYear

In [ ]:
%load_ext autoreload
%autoreload
from dbArtistsAlbumOfTheYear import dbArtistsAlbumOfTheYear
from dbArtistsParse import dbArtistsPrimary, dbArtistsCredit, dbArtistsExtra
from dbArtistsParse import dbArtistsAssertExtra, dbArtistsUnofficial, dbArtistsAssertUnofficial

In [ ]:
for modVal in range(100):
    dbAP = dbArtistsPrimary(dbArtistsAlbumOfTheYear())
    dometa = dbAP.parse(modVal, previousDays=10, force=False)
    if dometa:
        dbAP.createArtistMetadata(modVal)
        dbAP.createAlbumMetadata(modVal)

# Deezer

In [ ]:
%load_ext autoreload
%autoreload
from dbArtistsDeezer import dbArtistsDeezer
from dbArtistsParse import dbArtistsPrimary, dbArtistsCredit, dbArtistsExtra
from dbArtistsParse import dbArtistsAssertExtra, dbArtistsUnofficial, dbArtistsAssertUnofficial

In [ ]:
dbAP = dbArtistsPrimary(dbArtistsDeezer())
dometa = dbAP.parse(44, previousDays=1, force=True)
if dometa:
    dbAP.createArtistMetadata(44)
    dbAP.createAlbumMetadata(44)

In [ ]:
%load_ext autoreload
%autoreload
from dbArtistsDeezer import dbArtistsDeezer
from dbArtistsParse import dbArtistsPrimary, dbArtistsCredit, dbArtistsExtra
from dbArtistsParse import dbArtistsAssertExtra, dbArtistsUnofficial, dbArtistsAssertUnofficial
dbAP = dbArtistsDeezer(debug=True)

# MusicBrainz

In [ ]:
%load_ext autoreload
%autoreload
from dbArtistsMusicBrainz import dbArtistsMusicBrainz
from dbArtistsParse import dbArtistsPrimary, dbArtistsCredit, dbArtistsExtra
from dbArtistsParse import dbArtistsAssertExtra, dbArtistsUnofficial, dbArtistsAssertUnofficial
dbMB = dbArtistsMusicBrainz()
for modVal in range(2,100):
    #if True:
    #try:
        #lfm.assertDBModValExtraData(modVal, minPages=10, maxPages=20, test=False)
    dbMB.assertDBModValExtraData(modVal, minPages=2, maxPages=50, test=False, clean=True)
    #except:
    #    pass

# AllMusic

In [ ]:
%load_ext autoreload
%autoreload
from dbArtistsAllMusic import dbArtistsAllMusic
from dbArtistsParse import dbArtistsPrimary, dbArtistsCredit, dbArtistsExtra
from dbArtistsParse import dbArtistsAssertExtra, dbArtistsUnofficial, dbArtistsAssertUnofficial
from dbArtistsParse import dbArtistsAssertCredit, dbArtistsAssertComposition, dbArtistsAssertSong
dbAP = dbArtistsAllMusic(debug=True)

In [ ]:
artistDBData = dbAP.disc.getArtistsDBModValData(0)

In [ ]:
artistDBData['0000006000'].show()

In [ ]:
for artistID,artistData in artistDBData.items():
    print(artistID,'\t',artistData.media.media.keys())

In [ ]:
dbAP = dbArtistsAssertComposition(dbArtistsAllMusic())
dbAP.parseModVal(0)

In [ ]:
from ioUtils import getFile
data = getFile("/Users/tgadfort/Music/Discog/artists-allmusic-db/metadata/0-Metadata.p")

In [ ]:
{data

In [ ]:
dbAP = dbArtistsTest(dbArtistsAllMusic())
dbAP.parseModVal(0, force=True)

In [ ]:
suburl='tv-smiths-explorers-mn0000008802/discography'
if suburl.endswith("/discography"):
    suburl=suburl[:-12]

In [ ]:
dbAP = dbArtistsPrimary(dbArtistsAllMusic())
dometa = dbAP.parse(2, previousDays=None, force=True, debug=False)

In [ ]:
dbac = dbArtistsAssertCredit(dbArtistsAllMusic())
print(dbac.ignores.keys())

In [ ]:
dbAP = dbArtistsAssertCredit(dbArtistsAllMusic())
dbAP.parse(2, previousDays=None)

In [ ]:
for modVal in range(100):
    dbAP = dbArtistsPrimary(dbArtistsAllMusic())
    dometa = dbAP.parse(modVal, previousDays=60, force=False)
    dbAP.createArtistMetadata(modVal)
    dbAP.createAlbumMetadata(modVal)

In [ ]:
dbac = dbArtistsAssertComposition(dbArtistsAllMusic())
dbac.parseModVal(12)
dbac.downloadUnknownArtistCompositions()

In [ ]:
# This downloads missing credit files
for modVal in range(70,100):
    print(modVal)
    try:
        dbac = dbArtistsAssertCredit(dbArtistsAllMusic())
        dbac.parseModVal(modVal)
        dbac.downloadUnknownArtistCredits()
    except:
        continue


In [ ]:
%autoreload
from parseDBArtistsData import parseDBArtistsData
from dbArtistsParse import dbArtistsPrimary
from dbArtistsRateYourMusic import dbArtistsRateYourMusic

for modVal in range(1):
    dbAP = dbArtistsPrimary(dbArtistsRateYourMusic())
    dometa = dbAP.parse(modVal, previousDays=None, force=True, debug=False)
    #if dometa and False:
    #    dbAP.createArtistMetadata(modVal)
    #    dbAP.createAlbumMetadata(modVal)

In [ ]:


for modVal in range(100):
    dbAP = dbArtistsCredit(dbArtistsAllMusic())
    dbAP.parse(modVal, previousDays=30)
    

for modVal in range(100):
    dbAP = dbArtistsPrimary(dbArtistsAllMusic())
    dometa = dbAP.parse(modVal, previousDays=None, force=False)
    if dometa or True:
        dbAP.createArtistMetadata(modVal)
        dbAP.createAlbumMetadata(modVal)

In [ ]:
from ioUtils import getFile
x = getFile("/Users/tgadfort/Music/Discog/artists-deezer-db/metadata/5-MediaMetadata.p")

In [ ]:
x['5038705']

In [ ]:
modVal = 5
dbAP = dbArtistsPrimary(dbArtistsDeezer())
dometa = dbAP.parse(modVal, previousDays=1, force=True)
if dometa:
    dbAP.createArtistMetadata(modVal)
    dbAP.createAlbumMetadata(modVal)

In [ ]:
for modVal in range(100):
    dbAP = dbArtistsPrimary(dbArtistsDeezer())
    dometa = dbAP.parse(modVal, previousDays=1, force=True)
    if dometa:
        dbAP.createArtistMetadata(modVal)
        dbAP.createAlbumMetadata(modVal)

# Discogs

In [ ]:
%load_ext autoreload
%autoreload
from dbArtistsDiscogs import dbArtistsDiscogs
from dbArtistsParse import dbArtistsPrimary, dbArtistsCredit, dbArtistsExtra
from dbArtistsParse import dbArtistsAssertExtra, dbArtistsUnofficial, dbArtistsAssertUnofficial

In [ ]:
for modVal in range(100):
    dbAP = dbArtistsPrimary(dbArtistsDiscogs())
    dometa = dbAP.parse(modVal, previousDays=3, force=False)
    if dometa:
        dbAP.createArtistMetadata(modVal)
        dbAP.createAlbumMetadata(modVal)

In [ ]:
for modVal in range(100):
    dbAP = dbArtistsAssertUnofficial(dbArtistsDiscogs())
    dbAP.parseModVal(modVal)
    dbAP.downloadMissingArtistUnofficials()

# MusicBrainz

In [ ]:
%load_ext autoreload
%autoreload
from dbArtistsMusicBrainz import dbArtistsMusicBrainz
from dbArtistsParse import dbArtistsPrimary, dbArtistsCredit, dbArtistsExtra
from dbArtistsParse import dbArtistsAssertExtra

In [ ]:
for modVal in range(100):
    dbAP = dbArtistsPrimary(dbArtistsMusicBrainz())
    dometa = dbAP.parse(modVal, previousDays=10, force=False)
    if dometa:
        dbAP.createArtistMetadata(modVal)
        dbAP.createAlbumMetadata(modVal)

#for modVal in range(100):
#    dbAP = dbArtistsExtra(dbArtistsMusicBrainz())
#    dbAP.parse(modVal)

# LastFM

In [ ]:
%load_ext autoreload
%autoreload
from dbArtistsLastFM import dbArtistsLastFM
from dbArtistsParse import dbArtistsPrimary, dbArtistsCredit, dbArtistsExtra
from dbArtistsParse import dbArtistsAssertExtra, dbArtistsUnofficial, dbArtistsAssertUnofficial

In [ ]:
for modVal in range(100):
    dbAP = dbArtistsPrimary(dbArtistsLastFM())
    dometa = dbAP.parse(modVal, previousDays=10, force=False)
    if dometa:
        dbAP.createArtistMetadata(modVal)
        dbAP.createAlbumMetadata(modVal)

# KWorbSpotify

In [ ]:
%load_ext autoreload
%autoreload
from dbArtistsKWorbSpotify import dbArtistsKWorbSpotify
from dbArtistsParse import dbArtistsPrimary, dbArtistsCredit, dbArtistsExtra
from dbArtistsParse import dbArtistsAssertExtra, dbArtistsUnofficial, dbArtistsAssertUnofficial, dbArtistsRawFiles
from time import sleep

#dbAP = dbArtistsRawFiles(dbArtistsKWorbSpotify(), "data")
#dbAP.parse(previousDays=1000000)

In [ ]:
for modVal in range(100):
    dbAP = dbArtistsPrimary(dbArtistsKWorbSpotify())
    dometa = dbAP.parse(modVal, previousDays=3000, force=True, debug=False)
    if dometa:
        dbAP.createArtistMetadata(modVal)
        dbAP.createAlbumMetadata(modVal)
        
sleep(1)
start, cmt = clock("Creating DB")
maindb = mainDB(mdb=None, create=True, debug=True)
maindb.setDBFull("KWorbSpotify")
elapsed(start, cmt)    

# KWorbiTunes

In [ ]:
%load_ext autoreload
%autoreload
from dbArtistsKWorbiTunes import dbArtistsKWorbiTunes
from dbArtistsParse import dbArtistsPrimary, dbArtistsCredit, dbArtistsExtra
from dbArtistsParse import dbArtistsAssertExtra, dbArtistsUnofficial, dbArtistsAssertUnofficial, dbArtistsRawFiles
from time import sleep

#dbAP = dbArtistsRawFiles(dbArtistsKWorbiTunes(), "data")
#dbAP.parse(previousDays=1000000)

In [ ]:
for modVal in range(100):
    dbAP = dbArtistsPrimary(dbArtistsKWorbiTunes())
    dometa = dbAP.parse(modVal, previousDays=3000, force=True, debug=False)
    if dometa:
        dbAP.createArtistMetadata(modVal)
        dbAP.createAlbumMetadata(modVal)
        
sleep(1)
start, cmt = clock("Creating DB")
maindb = mainDB(mdb=None, create=True, debug=True)
maindb.setDBFull("KWorbiTunes")
elapsed(start, cmt)    

# KWorbYouTube

In [ ]:
# %autoreload
from dbArtistsKWorbYouTube import dbArtistsKWorbYouTube
dbAP = dbArtistsRawFiles(dbArtistsKWorbYouTube(), "data")
dbAP.parse(previousDays=10)

In [ ]:
%load_ext autoreload
%autoreload
from dbArtistsKWorbYouTube import dbArtistsKWorbYouTube
from dbArtistsParse import dbArtistsPrimary, dbArtistsCredit, dbArtistsExtra
from dbArtistsParse import dbArtistsAssertExtra, dbArtistsUnofficial, dbArtistsAssertUnofficial, dbArtistsRawFiles
from time import sleep

dbAP = dbArtistsRawFiles(dbArtistsKWorbYouTube(), "data")
#dbAP.parse(previousDays=1000000)

for modVal in range(100):
    dbAP = dbArtistsPrimary(dbArtistsKWorbYouTube())
    dometa = dbAP.parse(modVal, previousDays=3, force=True)
    if dometa:
        dbAP.createArtistMetadata(modVal)
        dbAP.createAlbumMetadata(modVal)
        
sleep(1)
start, cmt = clock("Creating DB")
maindb = mainDB(mdb=None, create=True, debug=True)
maindb.setDBFull("KWorbYouTube")
elapsed(start, cmt)    

In [ ]:
%autoreload
from dbArtistsRateYourMusic import dbArtistsRateYourMusic
#pdb = parseDBArtistsData()
dbAP = dbArtistsRawHTML(dbArtistsRateYourMusic())
dbAP.parse('< 5 days')

In [ ]:
%autoreload
from dbArtistsRateYourMusic import dbArtistsRateYourMusic
#pdb = parseDBArtistsData()
dbAP = dbArtistsPrimary(dbArtistsRateYourMusic())
for modVal in range(100):
    dbAP.parse(modVal=modVal, expr='< 0 hours')

## Assert Credit For AllMusic

In [ ]:
%load_ext autoreload
%autoreload
from dbArtistsAllMusic import dbArtistsAllMusic
from dbArtistsParse import dbArtistsPrimary, dbArtistsCredit, dbArtistsExtra
from dbArtistsParse import dbArtistsAssertExtra, dbArtistsUnofficial, dbArtistsAssertUnofficial, dbArtistsAssertCredit
dbAP = dbArtistsAllMusic(debug=True)
# This downloads missing credit files
ts = timestat("Getting Credit Files")
for modVal in range(20,30):
    try:
        dbac = dbArtistsAssertCredit(dbArtistsAllMusic())
        dbac.parseModVal(modVal)
        dbac.downloadUnknownArtistCredits()
    except:
        continue
        
ts.stop()

In [ ]:
%load_ext autoreload
%autoreload
from dbArtistsParse import dbArtistsPrimary, dbArtistsCredit, dbArtistsExtra, dbArtistsAssertCredit
from dbArtistsAllMusic import dbArtistsAllMusic

modVal=86
dbAC = dbArtistsAssertCredit(dbArtistsAllMusic())
dbAC.createCreditMetadata(modVal=modVal)
dbAC.downloadUnknownArtistCredits()

In [ ]:
dbC = dbArtistsCredit(dbArtistsAllMusic())
#dbC.parse(modVal=97, expr='< 90 Days', force=False)
dbC.parse(modVal=97, expr=None, force=False)

In [ ]:
%load_ext autoreload
%autoreload
from dbArtistsParse import dbArtistsPrimary, dbArtistsCredit, dbArtistsExtra, dbArtistsAssertCredit
from dbArtistsAllMusic import dbArtistsAllMusic

for modVal in range(80,81):
    dbAP = dbArtistsPrimary(dbArtistsAllMusic())
    _ = dbAP.parse(modVal=modVal, expr='< 0 Days', force=False)
    dbAP.createArtistMetadata(modVal)
    dbAP.createAlbumMetadata(modVal)
    
    dbAC = dbArtistsAssertCredit(dbArtistsAllMusic())
    dbAC.createCreditMetadata(modVal=modVal)
    dbAC.downloadUnknownArtistCredits()
    
    dbC = dbArtistsCredit(dbArtistsAllMusic())
    dbC.parse(modVal=modVal, expr='< 90 Days', force=False)
    dbAP.createArtistMetadata(modVal)
    dbAP.createAlbumMetadata(modVal)

In [ ]:
artistIDs=['0000047990', '0000051390', '0000272190', '0000292790', '0000467590', '0000602690', '0000717990', '0000796690', '0001033390', '0001378390', '0001407190', '0001464290', '0001995390', '0002323590', '0002425790', '0002765190', '0002879590', '0002903990', '0003093090', '0003190590', '0003246290', '0003286290', '0003918290']
artistIDs+=['0000118299', '0000213799', '0000325299', '0000642999', '0000775799', '0000811999', '0000992099', '0001284999', '0001364399', '0001497499', '0001510299', '0001618699', '0001634699', '0001658799', '0001694699', '0001711999', '0002179199', '0002366199', '0002612899', '0002752799', '0002987499', '0003027599', '0003184899', '0003258299', '0003442899', '0003678399']
len(artistIDs)

In [ ]:
%load_ext autoreload
%autoreload
from dbArtistsAllMusic import dbArtistsAllMusic
from dbArtistsParse import dbArtistsPrimary, dbArtistsCredit, dbArtistsExtra
from dbArtistsParse import dbArtistsAssertExtra, dbArtistsUnofficial, dbArtistsAssertUnofficial, dbArtistsAssertCredit
tmp = dbArtistsAssertCredit(dbArtistsAllMusic())
#tmp.updateMasterIgnoreCreditData(artistIDs)
#tmp.createCreditMetadata(modVal=98)
#tmp.downloadUnknownArtistCredits()

In [ ]:
from copy import deepcopy
meta = deepcopy(tmp.metadata)

In [ ]:
tmp.downloadUnknownArtistCredits()

In [ ]:
%load_ext autoreload
%autoreload
from dbArtistsAllMusic import dbArtistsAllMusic
from dbArtistsParse import dbArtistsPrimary, dbArtistsCredit, dbArtistsExtra
from dbArtistsParse import dbArtistsAssertExtra, dbArtistsUnofficial, dbArtistsAssertUnofficial, dbArtistsAssertCredit
tmp = dbArtistsCredit(dbArtistsAllMusic())
tmp.parse(modVal=99, expr='< 90 Days', force=False)

In [ ]:
from searchUtils import filesFromDir
from fsUtils import setDir
force=True
modVal=99
ffd   = filesFromDir(ext=".p")
files = ffd.getFiles(setDir(tmp.getModValDir(modVal), "credit"))
newFiles = files if force is True else tmp.getFilesByRecency(files, expr)

In [ ]:
%autoreload
from dbArtistsParse import dbArtistsRawHTML
from dbArtistsRateYourMusic import dbArtistsRateYourMusic
dbAP = dbArtistsRawHTML(dbArtistsRateYourMusic())
dbAP.parse(expr='< 3 Days')

In [ ]:
%autoreload
from dbArtistsParse import dbArtistsRawFiles
from dbArtistsKWorbiTunes import dbArtistsKWorbiTunes
dbAP = dbArtistsRawFiles(dbArtistsKWorbiTunes(), "data")
dbAP.parse(modVal=0, expr='< 3 Days')

In [ ]:
from searchUtils import filesFromDir
from fsUtils import setDir
ffd   = filesFromDir(ext=[".html", ".htm"])
files = ffd.getFiles(setDir(dbAP.disc.getArtistsDir(), "data"))
#newFiles = files if force is True else self.getFilesByRecency(files, expr)

In [ ]:
len(files)

In [ ]:
from recentFilesUtils import recentFiles
rf = recentFiles()
rf.setFiles(files)
newFiles = rf.getFilesByRecency('> 3 Days', debug=True)

In [ ]:
newFiles = dbAP.getFilesByRecency(files, expr='> 3 Days', debug=True)
len(newFiles)

In [ ]:
modValPrimaryFiles = tmp.getArtistPrimaryFiles(99, expr=None, force=True)
dbData = tmp.getDBData(99)

In [ ]:
modValArtistIDs  = {getBaseFilename(ifile): ifile for ifile in modValPrimaryFiles}
missingArtistIDs = [ifile for ifile in modValPrimaryFiles if dbData.get(getBaseFilename(ifile)) is None]
print(len(modValArtistIDs))
print(len(missingArtistIDs))

# Unofficial

In [ ]:
from dbArtistsBase import dbArtistsBase
from fsUtils import isFile
from fileUtils import getBaseFilename
from timeUtils import timestat
from ioUtils import saveFile
from time import sleep
import urllib
from webUtils import getHTML

In [ ]:
%autoreload

In [ ]:
%autoreload
from dbArtistsDiscogs import dbArtistsDiscogs
dbAP = dbArtistsAssertUnofficial(dbArtistsDiscogs())
dbAP.createUnofficialMetadata(99)
dbAP.downloadMissingArtistUnofficial()

In [ ]:
from dbArtistsParse import dbArtistsPrimary
dbAP = dbArtistsPrimary(dbArtistsDiscogs())
dbAP.parse(modVal=99, expr='< 90 Days', force=False)
dbAP

In [ ]:
dbAP = dbArtistsUnofficial(dbArtistsDiscogs())
dbAP.parse(modVal=99, expr='< 90 Days', force=True)

In [ ]:
%load_ext autoreload
%autoreload
from dbArtistsParse import dbArtistsPrimary
from dbArtistsParseUnofficial import dbArtistsAssertUnofficial, dbArtistsUnofficial
from dbArtistsDiscogs import dbArtistsDiscogs

for modVal in range(100):
    dbAC = dbArtistsAssertUnofficial(dbArtistsDiscogs())
    dbAC.createUnofficialMetadata(modVal=modVal)
    dbAC.downloadMissingArtistUnofficial()

In [ ]:
dbAC.downloadMissingArtistUnofficial()

In [ ]:
from dbArtistsParseUnofficial import dbArtistsAssertUnofficial, dbArtistsUnofficial

In [ ]:
#dbAP = dbArtistsPrimary(dbArtistsDiscogs())
#dbAP.createArtistMetadata(modVal)
#dbAP.createAlbumMetadata(modVal)
%load_ext autoreload
%autoreload
from dbArtistsParse import dbArtistsPrimary
from dbArtistsParseUnofficial import dbArtistsAssertUnofficial, dbArtistsUnofficial
from dbArtistsDiscogs import dbArtistsDiscogs

dbC = dbArtistsUnofficial(dbArtistsDiscogs())
dbC.parse(modVal=modVal, expr='< 1 Days', force=False)
dbC.createArtistMetadata(modVal)
dbC.createAlbumMetadata(modVal)

In [ ]:
len(ignoreData)

In [ ]:
len(unofficialFiles)

In [ ]:
%load_ext autoreload
%autoreload
from dbArtistsParse import dbArtistsPrimary
from dbArtistsParseUnofficial import dbArtistsAssertUnofficial, dbArtistsUnofficial
from dbArtistsDiscogs import dbArtistsDiscogs

for modVal in range(98,99):
    dbAP = dbArtistsPrimary(dbArtistsDiscogs())
    _ = dbAP.parse(modVal=modVal, expr='< 0 Days', force=False)
    dbAP.createArtistMetadata(modVal)
    dbAP.createAlbumMetadata(modVal)

    dbAC = dbArtistsAssertUnofficial(dbArtistsDiscogs())
    dbAC.createUnofficialMetadata(modVal=modVal)
    dbAC.downloadMissingArtistUnofficial()

    dbC = dbArtistsUnofficial(dbArtistsDiscogs())
    dbC.parse(modVal=modVal, expr='< 90 Days', force=True)
    dbC.createArtistMetadata(modVal)
    dbC.createAlbumMetadata(modVal)

# Extra

In [ ]:

from dbArtistsMusicBrainz import dbArtistsMusicBrainz
dbAP = dbArtistsAssertExtra(dbArtistsMusicBrainz())

In [ ]:
%load_ext autoreload
%autoreload
from dbArtistsParse import dbArtistsPrimary
from dbArtistsParseExtra import dbArtistsExtra, dbArtistsAssertExtra
from dbArtistsMusicBrainz import dbArtistsMusicBrainz

for modVal in range(72,73):
    if False:
        dbAP = dbArtistsPrimary(dbArtistsMusicBrainz())
        _ = dbAP.parse(modVal=modVal, expr='< 0 Days', force=False)
        dbAP.createArtistMetadata(modVal)
        dbAP.createAlbumMetadata(modVal)
        continue

    if False:
        dbAC = dbArtistsAssertExtra(dbArtistsMusicBrainz())
        dbAC.createExtraMetadata(modVal=modVal)
        dbAC.downloadMissingArtistExtras()
    
    if True:
        dbC = dbArtistsExtra(dbArtistsMusicBrainz())
        dbC.parse(modVal=modVal, expr='< 90 Days', force=True)
        dbAP.createArtistMetadata(modVal)
        dbAP.createAlbumMetadata(modVal)

In [ ]:
%load_ext autoreload
%autoreload
from dbArtistsParse import dbArtistsPrimary
from dbArtistsParseExtra import dbArtistsExtra, dbArtistsAssertExtra
from dbArtistsDiscogs import dbArtistsDiscogs

for modVal in range(72,73):
    if False:
        dbAP = dbArtistsPrimary(dbArtistsDiscogs())
        _ = dbAP.parse(modVal=modVal, expr='< 0 Days', force=False)
        dbAP.createArtistMetadata(modVal)
        dbAP.createAlbumMetadata(modVal)

    if True:
        dbAC = dbArtistsAssertExtra(dbArtistsDiscogs())
        dbAC.createExtraMetadata(modVal=modVal)
        dbAC.downloadMissingArtistExtras()

        dbC = dbArtistsExtra(dbArtistsDiscogs())
        dbC.parse(modVal=modVal, expr='< 90 Days', force=True)
        dbAP.createArtistMetadata(modVal)
        dbAP.createAlbumMetadata(modVal)

In [ ]:
%autoreload
from dbArtistsMusicBrainz import dbArtistsMusicBrainz
dbC = dbArtistsExtra(dbArtistsMusicBrainz())
dbC.parse(modVal=80, expr='< 90 Days', force=True)
dbAP.createArtistMetadata(80)
dbAP.createAlbumMetadata(80)

In [ ]:
dbAP.createExtraMetadata(72)

In [ ]:
from copy import deepcopy
meta = deepcopy(dbAP.metadata)

In [ ]:
dbAP.downloadMissingArtistExtras()

In [ ]:
ignoreData = dbAP.getMasterIgnoreData()

In [ ]:
ignoreData

In [ ]:
def getIgnoreArtistList():

    ###########################################################################################################################
    # Ignore List
    ############################################################################################################################
    ignoreArtistList = {}
    ignoreArtistList["Discogs"]  = ["M", "Bernard Edwards & Nile Rodgers", "Pomus-Shuman", "Tepper-Bennett"]
    ignoreArtistList["LastFM"]   = ["MAX", "Sonny Boy Williamson", "Trouble", "NRG", "Olivia", "Hunter", "Zeus", "Cobra", 
                                    "Shogun", "Nomad", "Juice", "Tyrant", "Willow", "Airbag", "Deuce", "Budgie", "Eclipse",
                                    "Jumbo", "L.E.J.", "Sniper", "Ceremony", "Voyager", "Solange", "Logos", 
                                    "The Highwaymen", "Rose", "Blake", "Tash", "Cast", "Friends", "Collage", "Ripe", 
                                    "Undercover", "Quest", "Holden"]

    ignoreArtistList["LastFM"]  += ["Íåèçâåñòíûé Èñïîëíèòåëü", "Coast To Coast AM - George Noory", 
                                    "[unknown]", "Various Composers", "[unknown]", "Zåìôèðà", 'Nature Sounds']
    ignoreArtistList["AllMusic"] = ["Bryan Adams", "Tina"]
    ignoreArtistList["MusicBrainz"] = ["Various Artists", "[unknown]", "[no artist]", "[language instruction]",
                                      "[nature sounds]", "Die drei ???", "[dialogue]", "初音ミク", '[data]', 
                                       'Geovanni', "Juan Torres", "DrefQuila", '[Disney]', 
                                       '[Christmas music]', 'Harry Nach', 'Tiago PZK']
    ignoreArtistList["RateYourMusic"] = []
    ignoreArtistList["Genius"] = []
    ignoreArtistList["KWorbiTunes"] = []
    ignoreArtistList["KWorbSpotify"] = []
    ignoreArtistList["Deezer"] = []
    ignoreArtistList["AlbumOfTheYear"] = []
    return ignoreArtistList

In [ ]:
for db,artistNames in getIgnoreArtistList().items():
    dbAP.updateMasterIgnoreNameData(db,artistNames)

In [ ]:
from dbArtistsLastFM import dbArtistsLastFM

In [ ]:
for modVal in range(1):
    dbAP = dbArtistsPrimary(dbArtistsLastFM())
    
    tsDBData = timestat("Finding Pages/URL Data For ModVal={0}".format(modVal))
    dbData = dbAP.getDBData(modVal)
    dbArtistURLPages = {artistID: {"Name": artistData.artist.name, "URL": artistData.url.url, "Pages": artistData.pages.pages} for artistID,artistData in dbData.items()}
    tsDBData.stop()
            
    tsPages = timestat("Finding Artists With More Pages From {0} Artists For ModVal={1}".format(len(dbArtistURLPages), modVal))
    pagesData = {artistID: artistData for artistID,artistData in dbArtistURLPages.items() if artistData["Pages"] is not None and artistData["Pages"] > 1}
    tsPages.stop()


In [ ]:
if False:
    dbAP = dbArtistsPrimary(dbArtistsLastFM())
    dbData = dbAP.getDBData(0)
    dbData['88336856000'].show()

if False:
    dbAP = dbArtistsPrimary(dbArtistsDiscogs())
    dbData = dbAP.getDBData(0)
    pages = dbData['15900'].pages

if True:
    dbAP = dbArtistsPrimary(dbArtistsMusicBrainz())
    dbData = dbAP.getDBData(0)
    pages = dbData['194279485768178399445457803447368290100'].pages
    
    

In [ ]:
pages.get()

In [ ]:
%autoreload
from dbArtistsParseExtra import dbArtistsAssertExtra
modVal=0
dbAC = dbArtistsAssertExtra(dbArtistsLastFM())
dbAC.createExtraMetadata(modVal=modVal)
dbAC.downloadMissingArtistExtras(maxPages=2)

In [ ]:
from copy import deepcopy
meta = deepcopy(dbAC.metadata)

In [ ]:
from pandas import DataFrame, concat
df = concat([DataFrame(metaD).T for metaD in meta.values()])

In [ ]:
dfS = df.sort_values(by="Pages", ascending=False)

In [ ]:
ignores =  ["Various Artists", "Kid Cudi vs Crookers", "Elvis Presley vs. Junkie XL", "Eric Prydz vs Floyd", "Rain Sounds", "Rest & Relax Nature Sounds Artists"]
ignores += ["Now That's What I Call Music!", "Best Relaxing Spa Music", "Relaxing Piano Music Consort"]
ignores

In [ ]:
dbAC.updateMasterIgnoreNameData(db="LastFM", artistNames=ignores)

In [ ]:
dfS[dfS["Name"].str.contains("Christmas")].head(30)